In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
import torch_tensorrt

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, cohen_kappa_score
from collections import deque
import pandas as pd
import math


import sys
sys.path.append('..')
from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

import minirocket_multi_numba


In [4]:
n = 2
np.array([(_ * ((np.sqrt(5) + 1) / 2)) % 1 for _ in range(1, n + 1)], dtype = np.float32)


array([0.618034  , 0.23606798], dtype=float32)

<h2> Build dataset dataframe <h2>


In [2]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['PIKACHU', 'PICHU'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

In [3]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())

PIKACHU    4096
PICHU       230
Name: labels, dtype: int64


/workspace/slp_jaspar/rocket_classifier_all_data/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


In [4]:
labels_order =  dataset.number_of_segments_per_game(3600,1000)
print(2**10)
print(labels_order)
labels_order = labels_order['Label'].values

1024
     Label  Count  Shift
0  PIKACHU   4067  30486
1    PICHU    227   1648


In [5]:
train_df, test_df  = dataset.train_test_split_dataframes(test_ratio = .20, val = False)
print(train_df.shape)
print(test_df.shape)

                           player_inputs_np_sub_path  length  num_segments  \
0  ranked\PIKACHU\da793855-4f9e-4f4f-a82a-1c559c9...   23242             1   
1  public\PIKACHU\2f695c31-7cc7-4003-ac07-e3a8132...   21865             1   
2  ranked\PIKACHU\1ed7d9c1-4f72-490d-857b-5bd056d...   21654             1   
3  ranked\PIKACHU\e4efdc4a-f484-403b-9b94-eba46b0...   20153             1   
4  ranked\PIKACHU\6966290b-b7c8-4eb1-8645-b84e31f...   19928             1   

    labels  encoded_labels  
0  PIKACHU               1  
1  PIKACHU               1  
2  PIKACHU               1  
3  PIKACHU               1  
4  PIKACHU               1  
(1600, 6)
(400, 6)


Build a label decoder for when we make charts to asses the model

In [6]:
labels_unique = train_df['labels'].unique()
encoded_labels_unique = train_df['encoded_labels'].unique()
label_decoder = zip(labels_unique, encoded_labels_unique)
label_decoder = dict(zip(encoded_labels_unique, labels_unique)) 
print(label_decoder)

{1: 'PIKACHU', 0: 'PICHU'}


<h2> Prepare the parameters of MiniRocket transformation </h2>

In [7]:
# Load a segment
with gzip.open('/workspace/melee_project_data/input_np/' + train_df['player_inputs_np_sub_path'][1000].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
segment = segment[:,0:3600]

segment[-5] = (segment[-5] > .5)
segment = np.expand_dims(segment,axis = 0)
# prepare minirocket transformation parameters
minirocket_transform_parameters = minirocket_multi_numba._fit_multi(segment)


<h2> Dataloader </h2>
We make a class that loads a segment and applies the minirocket transformation.

In [8]:
class TrainingDataset(Dataset):
    """
    Custom dataset for loading game segments from compressed numpy files.
    """
    def __init__(self, df, minirocket_transform_parameters, transform=None,):
        self.file_paths = df['player_inputs_np_sub_path'].to_numpy()
        self.encoded_labels = df['encoded_labels'].to_numpy()
        self.segment_start_index = df['segment_start_index'].to_numpy()
        # self.segment_index = df['segment_index'].to_numpy()
        self.segment_length = df['segment_length'].to_numpy()
        self.transform = transform
        self.parameters = minirocket_transform_parameters

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.file_paths)

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the specified index."""
        with gzip.open('/workspace/melee_project_data/input_np/' + self.file_paths[idx].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
        
        # Start and end of the segment
        segment_start = self.segment_start_index[idx]
        segment_end = self.segment_start_index[idx] + self.segment_length[idx]
        
        segment = segment[:,int(segment_start):int(segment_end)]
        
        segment[-5] = (segment[-5] > .5)    
            
        
        # Convert to PyTorch tensors
        # segment_transform = torch.from_numpy(segment).float()
        label_tensor = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return segment, label_tensor
    
def prepare_data_loaders(train_df, test_df, batch_size, num_workers):


    # Initialize datasets
    train_dataset = TrainingDataset(train_df,minirocket_transform_parameters,True)
    # val_dataset = TrainingDataset(file_paths_val, labels_val)
    test_dataset = TrainingDataset(test_df,minirocket_transform_parameters,True)

    # Initialize data loaders
    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True,persistent_workers=True, drop_last=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True, drop_last=True),
        # 'val': DataLoader(val_dataset, batch_size=2**9, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True)
    }
    return loaders


In [9]:
def train_model(model, criterion, optimizer, loaders, device, minirocket_transform_parameters, num_epochs=1):
    # Initialize progress bar for the epochs
    # epoch_progress = tqdm(range(num_epochs), desc='Training Progress', unit='epoch')
    for epoch in range(num_epochs):
        model.train()
        epoch_loss_sum = 0
        epoch_total = 0
        train_loader_tqdm = tqdm(loaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
        for input_cpu, target_cpu in train_loader_tqdm:
            # Move data to the appropriate device
            print(input_cpu.shape)
            transformed = torch.from_numpy(minirocket_multi_numba._transform_multi(input_cpu.numpy(), minirocket_transform_parameters))
            input_gpu = transformed.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            pred_gpu = model(input_gpu)
            # print(pred_gpu.shape)
            # Calculate loss
            loss = criterion(pred_gpu, target_cpu.to(device)) 
            loss.backward()
            
            # Clip gradients to avoid exploding gradients
            # clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Perform a single optimization step
            optimizer.step()
            
            # Accumulate loss sum for accurate average calculation
            # epoch_loss_sum += loss.item() * target_cpu.size(0)
            # epoch_total += target_cpu.size(0)

        # Calculate average loss for the epoch
        # epoch_loss = epoch_loss_sum / epoch_total

        # Update progress bar with the final loss of the epoch
        # epoch_progress.set_postfix(Loss=f'{epoch_loss:.10f}')

    return model


# def train_model(model, criterion, optimizer, loaders, device, num_epochs=1):
#     # Initialize progress bar for the epochs
#     epoch_progress = tqdm(range(num_epochs), desc='Training Progress', unit='epoch')
#     for epoch in epoch_progress:
#         model.train()
#         epoch_loss_sum = 0
#         epoch_total = 0

#         # Initialize the progress bar with auto-adjusting display and a higher refresh rate
#         train_loader_tqdm = tqdm(loaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch', dynamic_ncols=True, mininterval=1)
#         start_time = time.time()  # Start time for the first batch

#         for input_cpu, target_cpu in train_loader_tqdm:
#             # Calculate elapsed time and batches per second every time the loop runs
#             elapsed = time.time() - start_time
#             batches_per_second = (train_loader_tqdm.n / elapsed) if elapsed > 0 else 0
            
#             # Move data to the appropriate device
#             input_gpu = input_cpu.to(device)
            
#             # Zero the parameter gradients
#             optimizer.zero_grad()
            
#             # Forward pass
#             pred_gpu = model(input_gpu)
#             # Calculate loss
#             loss = criterion(pred_gpu, target_cpu.to(device))
#             loss.backward()
            
#             # Perform a single optimization step
#             optimizer.step()
            
#             # Accumulate loss sum for accurate average calculation
#             epoch_loss_sum += loss.item() * target_cpu.size(0)
#             epoch_total += target_cpu.size(0)

#             # Update the progress bar description with batches per second
#             train_loader_tqdm.set_description(f"Epoch {epoch+1}/{num_epochs} [{batches_per_second:.2f} batch/s]")

#         # Calculate average loss for the epoch
#         epoch_loss = epoch_loss_sum / epoch_total

#         # Update progress bar with the final loss of the epoch
#         epoch_progress.set_postfix(Loss=f'{epoch_loss:.10f}')

    return 


In [10]:
class Model(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Example usage
input_size = 9996
num_classes = train_df['labels'].unique().shape[0]  # Make sure this is defined in your code

model = Model(input_size, num_classes).to('cuda')

In [33]:
# default hyperparameters are reusable for any dataset
args = \
{
    "num_features"    : 10_000,
    "validation_size" : 2 ** 11,
    "chunk_size"      : 2 ** 12,
    "minibatch_size"  : 256,
    "lr"              : 1e-4,
    "max_epochs"      : 50,
    "patience_lr"     : 5,  #  50 minibatches
    "patience"        : 10, # 100 minibatches
    # "cache_size"      : training_size # set to 0 to prevent caching
}
# args = {**args, **kwargs}

_num_features = 84 * (args["num_features"] // 84)
# num_chunks = np.int32(np.ceil(training_size / args["chunk_size"]))

# -- model -----------------------------------------------------------------



loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 1e-4)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.5, min_lr = 1e-8, patience = 5)
model.to('cuda')

loaders = prepare_data_loaders(train_df, test_df, 400, 23)

In [34]:
train_model(model, loss_function, optimizer, loaders, 'cuda', minirocket_transform_parameters,  num_epochs=1)

Epoch 1/1:   0%|          | 0/4 [00:00<?, ?batch/s]

torch.Size([400, 9, 3600])


Epoch 1/1:  25%|██▌       | 1/4 [00:02<00:06,  2.29s/batch]

torch.Size([400, 9, 3600])


Epoch 1/1:  50%|█████     | 2/4 [00:03<00:03,  1.85s/batch]

torch.Size([400, 9, 3600])


Epoch 1/1:  75%|███████▌  | 3/4 [00:05<00:01,  1.69s/batch]

torch.Size([400, 9, 3600])


Epoch 1/1: 100%|██████████| 4/4 [00:06<00:00,  1.71s/batch]


Model(
  (model): Sequential(
    (0): Linear(in_features=9996, out_features=2, bias=True)
  )
)

In [30]:
train_model(model, loss_function, optimizer, loaders, 'cuda', minirocket_transform_parameters,  num_epochs=1)

Epoch 1/1:   0%|          | 0/1 [00:00<?, ?batch/s]

torch.Size([1600, 9, 3600])


Epoch 1/1: 100%|██████████| 1/1 [00:07<00:00,  7.71s/batch]


Model(
  (model): Sequential(
    (0): Linear(in_features=9996, out_features=2, bias=True)
  )
)

In [13]:
# Load a segment
with gzip.open('/workspace/melee_project_data/input_np/' + train_df['player_inputs_np_sub_path'][1000].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
segment = segment[:,0:3600]

segment[-5] = (segment[-5] > .5)
segment = np.expand_dims(segment,axis = 0)
# prepare minirocket transformation parameters
minirocket_transform_parameters = minirocket_multi_numba._fit_multi(segment)
minirocket_multi_numba._transform_multi(segment, minirocket_transform_parameters).shape
segment = np.squeeze(minirocket_multi_numba._transform_multi(segment, minirocket_transform_parameters),axis = 0)
segment_gpu = torch.from_numpy(segment).float().to('cuda')
segment_gpu.shape
model(segment_gpu)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)